## Is This Vinyl Scratched: Audio Classification

Dataset Source: https://www.kaggle.com/datasets/seandaly/detecting-scratch-noise-in-vinyl-playback

#### Install Missing Libraries

In [1]:
%pip install datasets transformers librosa evaluate tensorboard

Note: you may need to restart the kernel to use updated packages.


#### Import Necessary Libraries

In [2]:
import os, sys, random
os.environ['TOKENIZERS_PARALLELISM']='false'

import numpy as np
import pandas as pd

import datasets
from datasets import load_dataset, Audio, DatasetDict, ClassLabel

import transformers
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification
from transformers import TrainingArguments, Trainer

import evaluate

from IPython.display import Audio, display

#### Display Library Versions

In [3]:
print("Python:".rjust(15), sys.version[0:6])
print("NumPy:".rjust(15), np.__version__)
print("Pandas:".rjust(15), pd.__version__)
print("Datasets:".rjust(15), datasets.__version__)
print("Transformers:".rjust(15), transformers.__version__)
print("Evaluate:".rjust(15), evaluate.__version__)

        Python: 3.9.7 
         NumPy: 1.23.3
        Pandas: 1.4.4
      Datasets: 2.8.0
  Transformers: 4.26.0
      Evaluate: 0.2.2


#### Prepare Metadata File

In [4]:
image_file_names = os.listdir("/Users/leedunn/Desktop/Projects to Train/Audio Projects/scratched or not/")

metadata = pd.DataFrame(image_file_names, columns=["file_name"])

metadata['label'] = metadata['file_name'].apply(lambda x: x.split("sect")[-1].split(".wav")[0])
metadata['label'].filter(items=["0", "1"])

metadata.to_csv("/Users/leedunn/Desktop/Projects to Train/Audio Projects/scratched or not/metadata.csv", index=False)
metadata.head()

,file_name,label
0,358DSOTM sect0.wav,0
1,398GG sect1.wav,1
2,05DSOTM sect0.wav,0
3,451CR_RTH sect1.wav,1
4,323SW sect0.wav,0


#### Ingest & Preprocess Dataset

In [5]:
audio_data = load_dataset("/Users/leedunn/Desktop/Projects to Train/Audio Projects/scratched or not/", 
                          name="en-US", 
                          split="train")

audio_data = audio_data.filter(lambda example: example["label"] != None)
audio_data = audio_data.cast_column("label", ClassLabel(names=["0", "1"]))

print(len(audio_data))
print(audio_data)

Resolving data files:   0%|          | 0/3430 [00:00<?, ?it/s]

Using custom data configuration scratched or not-5a1b77f9dedd940e


Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset audiofolder downloaded and prepared to /Users/leedunn/.cache/huggingface/datasets/audiofolder/scratched or not-5a1b77f9dedd940e/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc. Subsequent calls will reuse this data.


  0%|          | 0/4 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

3428
Dataset({
    features: ['audio', 'label'],
    num_rows: 3428
})


#### Split Dataset into Training & Testing Datasets

In [6]:
audio_data = audio_data.shuffle(seed=42)

audio_data_split = audio_data.train_test_split(test_size=0.20)

ds = DatasetDict({
    'train' : audio_data_split['train'],
    'eval' : audio_data_split['test']
})

#### Some Information About Training & Validation Datasets

In [7]:
print("Training Dataset")
print("Training Dataset Info: ", ds['train'])
print("First Sample in Training Dataset", ds['train'][0])
print("Last Sample in Training Dataset", ds['train'][-1])
print("Unique Values in Label/Class: ", ds['train'].unique("label"))

print("\n\nEvaluation Dataset")
print("Evaluation Dataset Info: ", ds['eval'])
print("First Sample in Evaluation Dataset", ds['eval'][0])
print("Last Sample in Evaluation Dataset", ds['eval'][-1])
print("Unique Values in Label/Class: ", ds['eval'].unique("label"))

Training Dataset
Training Dataset Info:  Dataset({
    features: ['audio', 'label'],
    num_rows: 2742
})
First Sample in Training Dataset {'audio': {'path': '/Users/leedunn/Desktop/Projects to Train/Audio Projects/scratched or not/09CR_RTH sect1.wav', 'array': array([ 0.00808716,  0.01249695,  0.00984192, ..., -0.00941467,
       -0.00822449, -0.0078125 ], dtype=float32), 'sampling_rate': 22050}, 'label': 1}
Last Sample in Training Dataset {'audio': {'path': '/Users/leedunn/Desktop/Projects to Train/Audio Projects/scratched or not/366Rev sect1.wav', 'array': array([0.00476074, 0.00543213, 0.00109863, ..., 0.05975342, 0.04443359,
       0.01904297], dtype=float32), 'sampling_rate': 22050}, 'label': 1}


Flattening the indices:   0%|          | 0/3 [00:00<?, ?ba/s]

Unique Values in Label/Class:  [1, 0]


Evaluation Dataset
Evaluation Dataset Info:  Dataset({
    features: ['audio', 'label'],
    num_rows: 686
})
First Sample in Evaluation Dataset {'audio': {'path': '/Users/leedunn/Desktop/Projects to Train/Audio Projects/scratched or not/464DSOTM sect1.wav', 'array': array([-0.00311279, -0.01051331, -0.02253723, ..., -0.01145935,
       -0.01435852, -0.0118866 ], dtype=float32), 'sampling_rate': 22050}, 'label': 1}
Last Sample in Evaluation Dataset {'audio': {'path': '/Users/leedunn/Desktop/Projects to Train/Audio Projects/scratched or not/233DSOTM sect0.wav', 'array': array([ 0.02046204,  0.01290894, -0.00392151, ..., -0.0690918 ,
       -0.06637573, -0.06207275], dtype=float32), 'sampling_rate': 22050}, 'label': 0}


Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

Unique Values in Label/Class:  [1, 0]


#### Create Dictionaries to Convert Labels Between Strings & Integers

In [8]:
labels = sorted(set(audio_data["label"]))

print(labels)

label2id, id2label = dict(), dict()

for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

[0, 1]


#### Display Some Examples with Ability to Listen to Them

In [9]:
for _ in range(5):
    rand_idx = random.randint(0, len(ds["train"])-1)
    example = ds["train"][rand_idx]
    audio = example["audio"]
    
    print(f'Label: {id2label[str(example["label"])]}')
    print(f'Shape: {audio["array"].shape}, sampling rate: {audio["sampling_rate"]}')
    display(Audio(audio["array"], rate=audio["sampling_rate"]))
    print()

Label: 0
Shape: (88200,), sampling rate: 22050



Label: 0
Shape: (88200,), sampling rate: 22050



Label: 0
Shape: (88200,), sampling rate: 22050



Label: 0
Shape: (88200,), sampling rate: 22050



Label: 1
Shape: (88200,), sampling rate: 22050


#### Remember to Install git lfs & Enter HuggingFace Access Token

In [10]:
!git lfs install

# HuggingFace Access Token ...

Git LFS initialized.


#### Basic Values/Constants

In [11]:
MODEL_CKPT = "facebook/wav2vec2-base"
MODEL_NAME = MODEL_CKPT.split("/")[-1] + "-is_vinyl_scratched_or_not"

NUM_OF_EPOCHS = 10
LEARNING_RATE = 3e-5

BATCH_SIZE = 32
STRATEGY = "epoch"

#### Set Sample Rate

In [12]:
sampling_rate = ds["train"].features["audio"].sampling_rate
sampling_rate

#### Instantiate Instance of Feature Extractor

In [13]:
feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_CKPT)

/Users/leedunn/Documents/nlpnn/nlp/lib/python3.9/site-packages/transformers/configuration_utils.py:375: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


#### Define function to Preprocess Data

In [14]:
def preprocess_function(examples):
    '''
    This function prepares the dataset for the transformer
    by applying the feature extractor to it (among other 
    processes).
    '''
    max_duration = 5.5 # seconds
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(audio_arrays, 
                               sampling_rate=feature_extractor.sampling_rate, 
                               max_length=int(feature_extractor.sampling_rate * max_duration),
                               truncation=True)
    return inputs

encoded_audio = ds.map(preprocess_function, remove_columns="audio", batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

#### Define Metrics Evaluation Function 

In [15]:
def compute_metrics(p):
    '''
    This function calculates & returns the following metrics:
    - accuracy
    - f1 score
    - recall
    - precision
    '''
    import evaluate
    
    accuracy_metric = evaluate.load("accuracy")
    accuracy = accuracy_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)['accuracy']
    
    ### ------------------- F1 scores -------------------
    
    f1_score_metric = evaluate.load("f1")
    f1_score = f1_score_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)["f1"]
    
    ### ------------------- recall -------------------
    
    recall_metric = evaluate.load("recall")
    recall = recall_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)["recall"]
    
    ### ------------------- precision -------------------
    
    precision_metric = evaluate.load("precision")
    precision = precision_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)["precision"]
    
    return {"accuracy" : accuracy, 
            "F1" : f1_score,
            "Recall" : recall,
            "Precision" : precision,
            }

#### Instantiate Model

In [16]:
num_of_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(MODEL_CKPT, 
                                                        num_labels=num_of_labels, 
                                                        label2id=label2id,
                                                        id2label= id2label)

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForSequenceClassification: ['project_q.bias', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_hid.weight', 'quantizer.codevectors', 'project_hid.bias', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['projector.bias', 'projector.weight', 'classifier.

#### Define Training Arguments

In [17]:
args = TrainingArguments(
    output_dir=MODEL_NAME,
    evaluation_strategy=STRATEGY,
    num_train_epochs=NUM_OF_EPOCHS,
    save_strategy=STRATEGY,
    logging_strategy=STRATEGY,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_ratio=0.10,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
    metric_for_best_model="F1",
    greater_is_better=True,
    logging_first_step=True,
    report_to="tensorboard",
    hub_private_repo=True,
    push_to_hub=True
)

#### Define Trainer

In [18]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = encoded_audio["train"],
    eval_dataset = encoded_audio["eval"],
    tokenizer = feature_extractor,
    compute_metrics = compute_metrics,
)

Cloning https://huggingface.co/DunnBC22/wav2vec2-base-is_vinyl_scratched_or_not into local empty directory.


#### Train Model

In [19]:
trainer.train()

/Users/leedunn/Documents/nlpnn/nlp/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2742
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 210
  Number of trainable parameters = 94569090


  0%|          | 0/210 [00:00<?, ?it/s]

/Users/leedunn/Documents/nlpnn/nlp/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


{'loss': 0.6901, 'learning_rate': 1.4285714285714286e-06, 'epoch': 0.05}
{'loss': 0.6671, 'learning_rate': 3e-05, 'epoch': 0.98}


***** Running Evaluation *****
  Num examples = 686
  Batch size = 32


...

Saving model checkpoint to wav2vec2-base-is_vinyl_scratched_or_not/checkpoint-210
Configuration saved in wav2vec2-base-is_vinyl_scratched_or_not/checkpoint-210/config.json


{'eval_loss': 0.1040809378027916, 'eval_accuracy': 0.9752186588921283, 'eval_F1': 0.9637526652452025, 'eval_Recall': 0.9576271186440678, 'eval_Precision': 0.9699570815450643, 'eval_runtime': 1111.0181, 'eval_samples_per_second': 0.617, 'eval_steps_per_second': 0.02, 'epoch': 9.98}


Model weights saved in wav2vec2-base-is_vinyl_scratched_or_not/checkpoint-210/pytorch_model.bin
Feature extractor saved in wav2vec2-base-is_vinyl_scratched_or_not/checkpoint-210/preprocessor_config.json
Feature extractor saved in wav2vec2-base-is_vinyl_scratched_or_not/preprocessor_config.json
Several commits (2) will be pushed upstream.


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from wav2vec2-base-is_vinyl_scratched_or_not/checkpoint-168 (score: 0.9637526652452025).


{'train_runtime': 215824.6111, 'train_samples_per_second': 0.127, 'train_steps_per_second': 0.001, 'train_loss': 0.21470175442241488, 'epoch': 9.98}


TrainOutput(global_step=210, training_loss=0.21470175442241488, metrics={'train_runtime': 215824.6111, 'train_samples_per_second': 0.127, 'train_steps_per_second': 0.001, 'train_loss': 0.21470175442241488, 'epoch': 9.98})

#### Evaluate Model

In [20]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 686
  Batch size = 32


  0%|          | 0/22 [00:00<?, ?it/s]

{'eval_loss': 0.10392837971448898,
 'eval_accuracy': 0.9752186588921283,
 'eval_F1': 0.9637526652452025,
 'eval_Recall': 0.9576271186440678,
 'eval_Precision': 0.9699570815450643,
 'eval_runtime': 1035.8474,
 'eval_samples_per_second': 0.662,
 'eval_steps_per_second': 0.021,
 'epoch': 9.98}

#### Push Model to Hub (My Profile!!!)

In [26]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-base-is_vinyl_scratched_or_not
Configuration saved in wav2vec2-base-is_vinyl_scratched_or_not/config.json
Model weights saved in wav2vec2-base-is_vinyl_scratched_or_not/pytorch_model.bin
Feature extractor saved in wav2vec2-base-is_vinyl_scratched_or_not/preprocessor_config.json
Dropping the following result as it does not have all the necessary fields:
{'dataset': {'name': 'audiofolder', 'type': 'audiofolder', 'config': 'scratched or not', 'split': 'train', 'args': 'scratched or not'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9752186588921283}, {'name': 'F1', 'type': 'f1', 'value': 0.9637526652452025}, {'name': 'Recall', 'type': 'recall', 'value': 0.9576271186440678}, {'name': 'Precision', 'type': 'precision', 'value': 0.9699570815450643}]}


### Notes & Other Takeaways From This Project
****
- For some reason, it did not push the results of epoch 8 or 9 to the hub. I will attempt to manually upload that information.
- Results:
    - Accuracy: 0.9752186588921283
    - F1: 0.9637526652452025
    - Recall: 0.9576271186440678
    - Precision: 0.9699570815450643
****